## Import libraries

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from joblib import dump, load

from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans

from hmmlearn import hmm

## Data

In [ ]:
# Trawlers
Trawl = []
Trawl_l = []
for trawl in trawl_file:
    da = pd.read_csv("train/" + str(trawl) + ".csv")
    v = da["speed"].tolist()
    Trawl.append(v)
    Trawl_l.append(len(v))
    
# Seiners
Seiners = []
Seiners_l = []
for seiner in seiner_file:
    da = pd.read_csv("train/" + str(seiners) + ".csv")
    v = da["speed"].tolist()
    Seiners.append(v)
    Seiners_l.append(len(v))
    
# Gillnetters
Gill = []
Gill_l = []
for gill in gill_file:
    da = pd.read_csv("train/" + str(gill) + ".csv")
    v = da["speed"].tolist()
    Gill.append(v)
    Gill_l.append(len(v))

## Model

In [ ]:
# Using moving average to get each point's status
def mv(arr, window_size=3):
    
    h1 = arr.tolist().copy()
    h1.append(0)
    h1.append(0)

    h2 = arr.tolist().copy()
    h2.insert(0, 0)
    h2.append(0)

    h3 = arr.tolist().copy()
    h3.insert(0, 0)
    h3.insert(0, 0)

    H = np.array([h1,h2,h3])
    m = np.mean(H, axis=0)

    m[0] = arr[0]
    m[-1] = arr[-1]
    m[1] = (arr[0] + arr[1])/2
    m[-2] = (arr[-1] + arr[-2])/2
    return m

In [ ]:
# Training
Trawl_time = [np.lib.stride_tricks.sliding_window_view(trawl, window_shape=3) for trawl in Trawl]
Trawl_kmeans = KMeans(n_clusters=2, init='random',n_init=10, max_iter=300, tol=1e-04, random_state=0)
Trawl_kmeans.fit(np.concatenate(Trawl_time))
Trawl_results = mv(Trawl_kmeans.predict(data))